In [ ]:
import pandas as pd
df = pd.read_csv('clean_data.csv', lineterminator='\n')

In [ ]:
from random import randrange
from pprint import pprint

In [ ]:
x = randrange(0, len(df)-1)
pd.set_option('display.max_colwidth', None)
display(df[x:x+1])

In [ ]:
import spacy
from spacy import displacy
from pathlib import Path
nlp = spacy.load("en_core_web_sm")

In [3]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

In [6]:
from pathlib import Path
doc = nlp("Why did the chicken cross the road?")
svg = displacy.render(doc, style="dep", jupyter=False)

output_path = Path("dependency_plot.svg") # you can keep there only "dependency_plot.svg" if you want to save it in the same folder where you run the script 
output_path.open("w", encoding="utf-8").write(svg)

5507

In [ ]:
wordlistfile = open('wordlist.txt', 'r')
nonflatwordlist = [line.split() for line in wordlistfile.readlines()]
wordlist = [word for sublist in nonflatwordlist for word in sublist]

In [ ]:
wordlist[:5]

In [ ]:
wordlist.index('brother')

In [ ]:
from math import log10

In [ ]:
from copy import copy

In [ ]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')


In [ ]:
def templatify(text,deps, tags, ents):
    """
    The ratio between replacable and not-replaced tokens should be greater than or equal to 1:2
    Order of precedence is *nsubj > *pobj > *dobj > isolated adjectives > *verbs
    * denotes that it will include any immediate modifiers
    All named entities will be replaced
    """
    text = text.split()
    deps = deps.split()
    tags = tags.split()
    ents = ents.split()
    j = 0
    total_tokens = len(text)
    while True:
        if j == total_tokens - 1:
            break
        if tags[j]=='JJ':
            if tags[j+1]=='JJ' or tags[j+1][:2]=='NN':
                text.pop(j)
                deps.pop(j)
                tags.pop(j)
                ents.pop(j)
                j-=1
                total_tokens -= 1
        j+=1
    total_tokens = len(text)
    entities = 0
    weights = [0] * total_tokens
    for i in range(total_tokens-1):
        if tags[i][:2]=='NN' or tags[i][:2]=='VB' or tags[i][:2]=='JJ':
            if text[i] not in ['\'s','\'d'] and text[i] not in stopwords:
                entities+=1
    for i in range(total_tokens):
        """
        ne = 10
        nsubj = 5
        pobj = 4
        dobj = 3
        adj = 2
        verb = 1
        
        commonscore = log_10(rank) * 2.5
        
        final = posscore * commonscore
        """
        if tags[i][:2]=='NN':
            if deps[i]=='nsubj':
                weights[i] = 5
            if deps[i]=='pobj':
                weights[i] = 4
            else:
                weights[i] = 3
        elif tags[i][:2]=='JJ':
            weights[i]  = 2
        elif tags[i][:2]=='VB':
            weights[i] = 1
        if ents[i]!='None' and ents[i]!='CARDINAL':
            weights[i] = 10
        try:
            rank = wordlist.index(text[i])
            common_score = round(log10(rank+1)*2.5,2)
        except:
            common_score = weights[i]
        weights[i] *= common_score
    desired = entities/3
    template = copy(text)
    k = 0
    while k<=desired:
        i = weights.index(max(weights))
        weights[i] = -1
        template[i] = "[MASK]"
        k+=1
    #print(entities, total_tokens)     
    #print(text, weights)
    template = ' '.join(template)
    return template

In [ ]:
x = randrange(0, len(df)-1)
x = 49
print(x)
setup = df.at[x,'setup_text']
setup_dep = df.at[x,'setup_dep']
setup_tags = df.at[x,'setup_postags']
setup_ents = df.at[x,'setup_nertype']
punch = df.at[x,'punch_text']
punch_dep = df.at[x,'punch_dep']
punch_tags = df.at[x,'punch_postags']
punch_ents = df.at[x,'punch_nertype']
#print(setup, setup_tags)
setup_temp = templatify(setup, setup_dep, setup_tags, setup_ents)
punch_temp = templatify(punch, punch_dep, punch_tags, punch_ents)
print(setup_temp+' '+punch_temp)
# print(setup,'\n', punch, sep='')
# print(setup_dep,'\n', punch_dep, sep='')
# setup_parse = nlp(setup)
# punch_parse = nlp(punch)
# options = {"bg": "#ffffff"}
# displacy.render(setup_parse, style="dep", options=options)
# displacy.render(punch_parse, style="dep", options=options)

In [ ]:
all_templates = []

In [ ]:
for x, row in df.iterrows():
    setup = df.at[x,'setup_text']
    setup = setup.lower()
    setup_dep = df.at[x,'setup_dep']
    setup_tags = df.at[x,'setup_postags']
    setup_ents = df.at[x,'setup_nertype']
    punch = df.at[x,'punch_text']
    punch = punch.lower()
    punch_dep = df.at[x,'punch_dep']
    punch_tags = df.at[x,'punch_postags']
    punch_ents = df.at[x,'punch_nertype']
    setup_temp = templatify(setup, setup_dep, setup_tags, setup_ents)
    punch_temp = templatify(punch, punch_dep, punch_tags, punch_ents)
    masked_string = setup_temp + ' ' + punch_temp
    all_templates.append(masked_string)
    if x%5000==0:
        print(x)

In [ ]:
with open('my_templates.txt', 'w') as f:
    for item in all_templates:
        f.write("%s\n" % item)